In [1]:
%load_ext autoreload
%autoreload 2

In [67]:
import os
import cv2
from tqdm import tqdm
import shutil
import pandas as pd
from PIL import Image, ImageDraw

from face_detector import FaceDetector

In [73]:
IMAGES_DIR = 'fddb/fddb_images/'
ANNOTATIONS_PATH = 'fddb/fddb_folds/'
RESULT_DIR = 'fddb/result/'
ANNOTATED_IMAGES_DIR = RESULT_DIR + 'ann_images/'
MODEL_PATH = 'model.pb'

# Collect annotated images

In [4]:
annotations = [s for s in os.listdir(ANNOTATIONS_PATH) if s.endswith('ellipseList.txt')]
image_lists = [s for s in os.listdir(ANNOTATIONS_PATH) if not s.endswith('ellipseList.txt')]
annotations = sorted(annotations)
image_lists = sorted(image_lists)

In [5]:
# combine all images pathes of FDDB that exist in different files of fddb_folds directory into one list
images_to_use = []
for n in image_lists:
    with open(os.path.join(ANNOTATIONS_PATH, n)) as f:
        images_to_use.extend(f.readlines())

In [7]:
print(images_to_use[0:5])

['2002/08/11/big/img_591\n', '2002/08/26/big/img_265\n', '2002/07/19/big/img_423\n', '2002/08/24/big/img_490\n', '2002/08/31/big/img_17676\n']


In [9]:
images_to_use = [s.strip() for s in images_to_use]
print(images_to_use[0:5])

if not os.path.exists(RESULT_DIR):
    os.mkdir(RESULT_DIR)

# write all the combined images pathes into ine txt file
with open(os.path.join(RESULT_DIR, 'faceList.txt'), 'w') as f:
    for p in images_to_use:
        f.write(p + '\n')

['2002/08/11/big/img_591', '2002/08/26/big/img_265', '2002/07/19/big/img_423', '2002/08/24/big/img_490', '2002/08/31/big/img_17676']


In [10]:
# combine all ellipses annotations of FDDB that exist in different files of fddb_folds directory into one list
ellipses = []
for n in annotations:
    with open(os.path.join(ANNOTATIONS_PATH, n)) as f:
        ellipses.extend(f.readlines())

In [11]:
i = 0
# write all the combined ellipses annotations into ine txt file
with open(os.path.join(RESULT_DIR, 'ellipseList.txt'), 'w') as f:
    for p in ellipses:
        
        # check image order
        if 'big/img' in p:
            assert images_to_use[i] in p
            i += 1

        f.write(p)

# Predict using trained detector

In [12]:
face_detector = FaceDetector(MODEL_PATH, gpu_memory_fraction=0.25, visible_device_list='0')

In [37]:
predictions = []
for n in tqdm(images_to_use):
    image_array = cv2.imread(os.path.join(IMAGES_DIR, n) + '.jpg')
    image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
    # threshold is important to set low
    boxes, scores = face_detector(image_array, score_threshold=0.5)
    predictions.append((n, boxes, scores))

100%|██████████| 2845/2845 [02:00<00:00, 23.57it/s]


In [40]:
with open(os.path.join(RESULT_DIR, 'detections_2.txt'), 'w') as f:
    for n, boxes, scores in predictions:
        f.write(n + '\n')
        f.write(str(len(boxes)) + '\n')
        for b, s in zip(boxes, scores):
            ymin, xmin, ymax, xmax = b
            h, w = int(ymax - ymin), int(xmax - xmin)
            f.write('{0} {1} {2} {3} {4:.4f}\n'.format(int(xmin), int(ymin), w, h, s))

# Copy images

In [15]:
for n in tqdm(images_to_use):
    p = os.path.join(RESULT_DIR, 'images', n + '.jpg')
    os.makedirs(os.path.dirname(p), exist_ok=True)
    shutil.copy(os.path.join(IMAGES_DIR, n) + '.jpg', p)

100%|██████████| 2845/2845 [00:07<00:00, 399.13it/s]


## Visualize the results of boxes on the images

save the detection results in a dataframe

In [77]:
detections = pd.DataFrame()
num_imgs = 0
with open(os.path.join(RESULT_DIR, 'detections_2.txt'), 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break

        image = line.strip() +  '.jpg'

        num_imgs += 1
        num_boxes = int(f.readline().strip())
        
        boxes_of_boxes = []
        boxes = []
        for i in range(num_boxes):
            xmin, ymin, w, h, s = f.readline().strip().split(" ")
            box = [int(xmin), int(ymin), int(xmin)+int(w), int(ymin)+int(h), float(s)]
            boxes.append(box)
            # boxes.extend(box)
        boxes_of_boxes.append(boxes)
        df = {'image_path': image, 'boxes': boxes_of_boxes} 
        df = pd.DataFrame(df)
        detections =  pd.concat([detections, df], ignore_index = True, axis = 0)

                   image_path  \
0  2002/08/26/big/img_265.jpg   

                                               boxes  
0  [[61, 39, 148, 155, 0.9794], [299, 55, 388, 18...  


In [70]:
detections.size
# print(image)
# print(boxes)
print(num_imgs)
print(len(boxes))

2845
1


In [78]:
detections.head()

,image_path,boxes
0,2002/08/11/big/img_591.jpg,"[[193, 59, 340, 263, 0.9856]]"
1,2002/08/26/big/img_265.jpg,"[[61, 39, 148, 155, 0.9794], [299, 55, 388, 18..."
2,2002/07/19/big/img_423.jpg,"[[203, 63, 307, 210, 0.9867]]"
3,2002/08/24/big/img_490.jpg,"[[105, 37, 176, 132, 0.9286]]"
4,2002/08/31/big/img_17676.jpg,"[[62, 40, 158, 169, 0.9741], [2, 13, 60, 101, ..."


In [120]:
def draw_boxes_on_image(path, boxes):

    image_copy = Image.open(path)
    draw = ImageDraw.Draw(image_copy)

    for b in boxes:
        xmin, ymin, xmax, ymax, s = b
        outline = 'red'
        draw.rectangle(
            [(xmin, ymin), (xmax, ymax)],
             outline=outline
        )
        draw.text((xmin, ymin), text='{:.3f}'.format(s))
    return image_copy


In [118]:
detections.tail()

,image_path,boxes
2840,2002/08/05/big/img_3704.jpg,"[[104, 35, 188, 166, 0.9813], [336, 122, 409, ..."
2841,2002/08/07/big/img_1358.jpg,"[[86, 70, 242, 290, 0.9555]]"
2842,2002/07/22/big/img_306.jpg,"[[125, 48, 202, 151, 0.9493]]"
2843,2002/08/13/big/img_619.jpg,"[[102, 121, 168, 210, 0.9613], [273, 119, 346,..."
2844,2002/08/02/big/img_366.jpg,"[[38, 43, 264, 352, 0.9928]]"


In [128]:

for index, row in detections.iterrows():
    img_path = row['image_path']
    image = IMAGES_DIR + img_path
    annotated_image = draw_boxes_on_image(image, row['boxes'])
    
    output_path = img_path.replace("/", "_")
    output_path = ANNOTATED_IMAGES_DIR + output_path
    annotated_image.save(output_path)

HH


In [129]:
files = os.listdir(ANNOTATED_IMAGES_DIR)
len(files)

2846